In [1]:
import numpy as np
import pandas as pd

c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Load data
Align everything to the LEFT video

In [2]:
pid = '03c42ea1-1e04-4a3e-9b04-46d8568dcd02'
VIDEO_DIR = 'D:/ibl_website_videos/proc'
DATA_DIR = './data'

In [3]:
# load timestamp files
body_ts = np.load(f'{DATA_DIR}/{pid}/{pid}_body_times_scaled.npy')
left_ts = np.load(f'{DATA_DIR}/{pid}/{pid}_left_times_scaled.npy')
right_ts = np.load(f'{DATA_DIR}/{pid}/{pid}_right_times_scaled.npy')

In [13]:
len(body_ts)

124674

In [4]:
# Load csv files
dlc_right = pd.read_csv(f'{DATA_DIR}/{pid}/{pid}_right_dlc_scaled.csv')
dlc_left = pd.read_csv(f'{DATA_DIR}/{pid}/{pid}_left_dlc_scaled.csv')
dlc_body = pd.read_csv(f'{DATA_DIR}/{pid}/{pid}_body_dlc_scaled.csv')

In [16]:
dlc_body

,Unnamed: 0,tail_start_x,tail_start_y,tail_start_likelihood
0,0,69.575142,71.461617,0.766433
1,1,70.935722,71.676584,0.979242
2,2,71.095446,71.676584,0.975016
3,3,71.223619,71.676584,0.991126
4,4,71.495390,71.614178,0.999873
...,...,...,...,...
124669,124669,70.750483,72.055346,0.999785
124670,124670,70.676630,72.055346,0.999842
124671,124671,70.630869,72.055346,0.999141
124672,124672,70.623337,72.055346,0.999672


In [5]:
# Load wheel
wheel = np.load(f'{DATA_DIR}/{pid}/{pid}_left_wheel_scaled.npy')

# Re-indexing
We're going to re-index everything to be matched to the *LEFT* video timestamps. So those timestamps will be in real-time but the body/right video will just tell you which frame corresponds to the left video frame.

(1) re-index videos, dlc, and wheel
(2) re-index trial events

In [6]:
def flatten(list_of_list):
  ret = []
  for list in list_of_list:
    ret += list
  return ret

In [7]:
# we will also be adding all the DLC points to this, we'll get them automatically and we'll get both the _x and _y coordinates for each
dlc_body_keys = ['tail_start']
dlc_left_keys = ['cl_nose_tip', 'cl_paw_l', 'cl_paw_r', 'cl_tube_top', 'cl_tongue_end_l', 'cl_tongue_end_r']
dlc_right_keys = ['cr_nose_tip', 'cr_paw_l', 'cr_paw_r', 'cr_tube_top', 'cr_tongue_end_l', 'cr_tongue_end_r']
# we'll handle pupil separately because it was cropped out
dlc_left_pupil_keys = ['pupil_right_r', 'pupil_left_r', 'pupil_top_r', 'pupil_bottom_r']

# generate the x/y versions
dlc_body_keys_xy = flatten([(x + '_x', x + '_y') for x in dlc_body_keys])
dlc_left_keys_xy = flatten([(x + '_x', x + '_y') for x in dlc_left_keys])
dlc_right_keys_xy = flatten([(x + '_x', x + '_y') for x in dlc_right_keys])
dlc_left_pupil_keys_xy = flatten([(x + '_x', x + '_y') for x in dlc_left_pupil_keys])

# 1 re-index videos, dlc, and wheel into one file
indexes = pd.DataFrame(columns=['left_ts','right_idx','body_idx','wheel'] + dlc_body_keys_xy + dlc_left_keys_xy + dlc_right_keys_xy + dlc_left_pupil_keys_xy)

In [8]:
print(indexes.keys())

Index(['left_ts', 'right_idx', 'body_idx', 'wheel', 'tail_start_x',
       'tail_start_y', 'cl_nose_tip_x', 'cl_nose_tip_y', 'cl_paw_l_x',
       'cl_paw_l_y', 'cl_paw_r_x', 'cl_paw_r_y', 'cl_tube_top_x',
       'cl_tube_top_y', 'cl_tongue_end_l_x', 'cl_tongue_end_l_y',
       'cl_tongue_end_r_x', 'cl_tongue_end_r_y', 'cr_nose_tip_x',
       'cr_nose_tip_y', 'cr_paw_l_x', 'cr_paw_l_y', 'cr_paw_r_x', 'cr_paw_r_y',
       'cr_tube_top_x', 'cr_tube_top_y', 'cr_tongue_end_l_x',
       'cr_tongue_end_l_y', 'cr_tongue_end_r_x', 'cr_tongue_end_r_y',
       'pupil_right_r_x', 'pupil_right_r_y', 'pupil_left_r_x',
       'pupil_left_r_y', 'pupil_top_r_x', 'pupil_top_r_y', 'pupil_bottom_r_x',
       'pupil_bottom_r_y'],
      dtype='object')


In [9]:
def get_dlc_coord(row, key):
  val = row[key]
  if np.isnan(val):
    val = -1
  return val

In [17]:
data_cols = {}

crop_data = pd.read_csv(f'{DATA_DIR}/{pid}/{pid}_left_pupil_rect.csv')

pupil_xy = [crop_data.x0_ss[0], crop_data.y0_ss[0]]

suffixes = ['_x', '_y']

for key in dlc_body_keys + dlc_left_keys + dlc_right_keys + dlc_left_pupil_keys:
  data_cols[key] = []

for i, lts in enumerate(left_ts):
  # get indexes
  right_idx = np.argmin(np.abs(right_ts-lts))
  body_idx = np.argmin(np.abs(body_ts-lts))

  row_data = [lts, right_idx, body_idx, wheel[i]]

  # pull the data columns
  for key in dlc_body_keys:
    for suffix in suffixes:
      row_data.append(get_dlc_coord(dlc_body.iloc[body_idx], key + suffix))
  for key in dlc_left_keys:
    for suffix in suffixes:
      row_data.append(get_dlc_coord(dlc_left.iloc[i], key[3:] + suffix))
  for key in dlc_right_keys:
    for suffix in suffixes:
      row_data.append(get_dlc_coord(dlc_right.iloc[right_idx], key[3:] + suffix))

  # get the pupil, and offset by the x0/y0 values
  for key in dlc_left_pupil_keys:
    for j, suffix in enumerate(suffixes):
      val = get_dlc_coord(dlc_left.iloc[i], key + suffix) - pupil_xy[j]
      row_data.append(val)
  
  indexes.loc[i] = row_data

In [18]:
indexes   

,left_ts,right_idx,body_idx,wheel,tail_start_x,tail_start_y,cl_nose_tip_x,cl_nose_tip_y,cl_paw_l_x,cl_paw_l_y,...,cr_tongue_end_r_x,cr_tongue_end_r_y,pupil_right_r_x,pupil_right_r_y,pupil_left_r_x,pupil_left_r_y,pupil_top_r_x,pupil_top_r_y,pupil_bottom_r_x,pupil_bottom_r_y
0,19.815952,1.0,0.0,-4.890537,69.575142,71.461617,30.062249,25.769378,52.119918,74.086943,...,128.811271,47.015935,64.588805,14.658144,63.089701,14.834160,64.166320,13.180284,64.551486,15.397514
1,19.849248,1.0,0.0,-4.891517,69.575142,71.461617,30.118009,25.865307,52.124124,74.086943,...,128.811271,47.015935,65.711953,14.988984,63.335984,15.948072,64.361644,13.184779,64.723242,15.891717
2,19.882545,2.0,0.0,-4.892895,69.575142,71.461617,30.522107,24.621611,52.124124,74.025011,...,128.426812,47.051358,65.695178,14.931786,63.618937,15.043428,64.425977,13.859271,64.767976,15.517295
3,19.915842,3.0,1.0,-4.892448,70.935722,71.676584,30.522107,24.336157,52.018185,73.888397,...,128.906451,46.912774,65.647777,14.526963,63.678035,15.027560,64.574659,13.965688,64.625696,15.449693
4,19.949138,4.0,2.0,-4.886652,71.095446,71.676584,30.551573,24.513149,51.163998,73.515120,...,128.737867,47.048504,65.574868,14.742753,63.678035,15.043428,64.579036,13.965688,64.625696,15.916952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124195,4155.036521,103660.0,124668.0,434.287430,70.750483,72.125237,28.607102,26.400964,54.284525,74.542851,...,130.419512,46.677726,65.587950,14.572087,63.222960,14.485680,64.021490,12.878407,63.706400,15.639757
124196,4155.069818,103661.0,124669.0,434.287194,70.750483,72.055346,28.538704,26.454173,54.284091,74.547062,...,130.410292,46.676765,65.587950,14.577095,63.249850,14.461069,64.020873,12.873804,63.701715,15.631251
124197,4155.103115,103661.0,124670.0,434.286959,70.676630,72.055346,28.532066,26.552836,54.280210,74.550071,...,130.410292,46.676765,65.593704,14.577095,63.249850,14.462563,64.026176,12.873804,63.701715,15.631251
124198,4155.136411,103662.0,124671.0,434.286743,70.630869,72.055346,28.531879,26.561681,54.272590,74.554734,...,130.408463,46.669268,65.593704,14.573150,63.251801,14.454357,64.028313,12.879293,63.714785,15.676721


In [21]:
for key in indexes.keys():
  dat = indexes[key].values.astype(np.float32)
  with open(f'{DATA_DIR}/{pid}/{pid}.{key}.bytes', 'wb') as file:
    file.write(dat.tobytes())

In [67]:
trials

,start_timestamp,stim_on_timestamp,feedback_timestamp,stimulus_side,contrast,feedback
0,118.805932,131.558813,131.486621,L,1.0000,-1.0
1,134.442952,135.725805,136.128911,R,0.1250,1.0
2,138.153175,139.213595,139.483932,L,0.1250,1.0
3,141.504452,142.841833,143.274218,L,1.0000,1.0
4,145.269490,146.713432,148.392382,L,0.2500,1.0
...,...,...,...,...,...,...
420,2560.430995,2560.915221,2620.917565,R,0.0625,-1.0
421,2624.380216,2624.436843,NaN,L,0.1250,1.0
422,2677.731984,NaN,NaN,R,0.0625,-1.0
423,2685.686023,NaN,NaN,R,0.2500,-1.0


In [69]:
trials

,start_timestamp,stim_on_timestamp,feedback_timestamp,stimulus_side,contrast,feedback
0,0.0,90.0,88.0,L,1.0000,-1.0
1,177.0,215.0,228.0,R,0.1250,1.0
2,288.0,320.0,328.0,L,0.1250,1.0
3,389.0,429.0,442.0,L,1.0000,1.0
4,502.0,545.0,596.0,L,0.2500,1.0
...,...,...,...,...,...,...
420,73038.0,73052.0,74854.0,R,0.0625,-1.0
421,74958.0,74960.0,0.0,L,0.1250,1.0
422,76560.0,0.0,0.0,R,0.0625,-1.0
423,76799.0,0.0,0.0,R,0.2500,-1.0


In [70]:
trials.to_csv(f'{DATA_DIR}/{pid}/{pid}.trials.csv', index = False)